In [ ]:
import os
import numpy as np
import pandas as pd
import jinja2 as jj


In [ ]:
def mklbl(prefix, n):
    return ["%s%s" % (prefix, i) for i in range(n)]

miindex = pd.MultiIndex.from_product([mklbl('A', 4),
                                      mklbl('B', 2),
                                      mklbl('C', 4),
                                      mklbl('D', 2)],
                                    names=['RowIdx-1', 'RowIdx-2', 'RowIdx-3', 'RowIdx-4'])
index  =['-'.join(col).strip() for col in miindex.values]
micolumns = pd.MultiIndex.from_tuples([('a', 'foo', 'zap'),
                                       ('a', 'foo', 'zip'),
                                       ('a', 'bar', 'zap'),
                                       ('a', 'bar', 'zip'),
                                       ('b', 'foo', 'zap'),
                                       ('b', 'foo', 'zep'),
                                       ('b', 'bah', 'zep'),
                                       ('b', 'bah', 'zyp'),
                                       ('b', 'bah', 'zap'),
                                      ],
                                      names=['ColIdx-{}'.format(i) for i in range(1, 4)])
cols  =['-'.join(col).strip() for col in micolumns.values]
data = np.arange(len(miindex) * len(micolumns), dtype=np.float).reshape((len(miindex),len(micolumns)))
data = data.tolist()


dfrc = pd.DataFrame(data, index=miindex, columns=micolumns).sort_index().sort_index(axis=1)

dfr  = pd.DataFrame(data, index=miindex, columns=cols).sort_index().sort_index(axis=1)
dfr.columns.name = 'UniqueCol'

dfc  = pd.DataFrame(data, index=index, columns=micolumns).sort_index().sort_index(axis=1)
dfc.index.name = 'UniqueRow'

df  = pd.DataFrame(data, index=index, columns=cols).sort_index()
df.index.name = 'UniqueRow'
df.columns.name = 'UniqueCol'

dfrc.info()
dfr.info()
dfc.info()
df.info()

In [ ]:
dfrc.head()

## Save df html
Must:
+ use notebook.css
+ wrap dataframe.html in specific classes - like in notebook
Result can be iframed in any doc


In [ ]:
%%writefile templates/index.tpl.html

<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <title>dataframe</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="https://cdn.jupyter.org/notebook/5.6.0/style/style.min.css">
</head>

<body>


    <div class="output_are">
        <div class="output_subarea output_html rendered_html output_result">

            __$data.df_html$__

        </div>
    </div>

</body>

</html>

In [ ]:
dir_template = 'templates'
dir_dump = 'dump'

loader = jj.FileSystemLoader(dir_template)
env = jj.Environment(loader=loader,
                     variable_start_string='__$',
                     variable_end_string='$__',
                     block_start_string='{-%',
                     block_end_string='%-}'
                     )

template = env.get_template('index.tpl.html')
# data = {'df_html': dfrc.to_html()}
data = {'df_html': dfrc.head(10).to_html()}
content = template.render(data=data)

if not os.path.exists(dir_dump):
    os.makedirs(dir_dump)

path = os.path.join(dir_dump, 'index.html')
with open(path, 'w') as f:
    f.write(content)
    print('file {} saved to disk'.format(path))


In [ ]:
!cd dump && python -m http.server 8080